# Ejercicio 1

Manuel tiene un local de pastas en Tafí Viejo. Acaba de firmar un presupuesto para
vender pastas en un casamiento. Para ello necesita:
- 20 kgs de carne
- 50 kgs de harina
- 30 kgs de tomate 
- 30 kgs de queso

Tiene dos proveedores:
- Distribuidora La Pedrera, que le puede vender 
    - Hasta 15 kgs de carne a $3000 el kg
    - Harina a $650 el kg
    - Tomate a $1300 el kg
    - Queso a $1800 el kg
- Distribuidora El Nogal, que le puede vender
    - Carne a $3200 el kg
    - Hasta 10kg de harina a $550 el kg
    - Tomate a $1200 el kg
    - Queso a $1900 el kg

Por otro lado, la cantidad máxima que se puede comprar a un único proveedor está limitada a 80kg, debido a las restricciones de capacidad en los camiones que utilizan un solo viaje por proveedor.

Además, tiene un acuerdo en el que por lo menos debe gastar $60000 en la distribuidora La Pedrera

In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

# Con camión de 80 kg


In [2]:
solver = pywraplp.Solver.CreateSolver('GLOP')

### Variables de decisión

Para este ejercicio, las variables de decisión son los kg de cada insumo en cada proveedor.

In [3]:
variables = {}
for proveedor in ['La Pedrera', 'El Nogal']:
    for insumo in ['carne', 'harina', 'tomate', 'queso']:
        var = solver.NumVar(0, solver.infinity(), f'Cantidad de kilos de {insumo} del proveedor {proveedor}')
        variables[(proveedor, insumo)] = var

### Restricciones

In [4]:
for proveedor in ['La Pedrera', 'El Nogal']:
    solver.Add(
        variables[(proveedor, 'carne')] + 
        variables[(proveedor, 'harina')] + 
        variables[(proveedor, 'tomate')] + 
        variables[(proveedor, 'queso')] <= 80, 
        f'Maxima cantidad que puedo comprar al proveedor {proveedor}'
    ) 


solver.Add(variables[('La Pedrera', 'carne')] + variables[('El Nogal', 'carne')] == 20, 'Maxima cantidad de kg de carne total.')
solver.Add(variables[('La Pedrera', 'harina')] + variables[('El Nogal', 'harina')] == 50, 'Maxima cantidad de kg de harina total.')
solver.Add(variables[('La Pedrera', 'tomate')] + variables[('El Nogal', 'tomate')] == 30, 'Maxima cantidad de kg de tomate total.')
solver.Add(variables[('La Pedrera', 'queso')] + variables[('El Nogal', 'queso')] == 30, 'Maxima cantidad de kg de queso total.')


solver.Add(variables[('La Pedrera', 'carne')]  <= 15, 'Maxima cantidad de kg de carne de La Pedrera.')
solver.Add(variables[('El Nogal', 'harina')]  <= 10, 'Maxima cantidad de kg de harina de El Nogal.')

acuerdo_pedrera = solver.Add(
    3000*variables[('La Pedrera', 'carne')] + 
    650*variables[('La Pedrera', 'harina')] + 
    1300*variables[('La Pedrera', 'tomate')] +
    1800*variables[('La Pedrera', 'queso')] >= 60000, 'Acuerdo con La Pedrera')


### Función objetivo

La función objetivo es el costo total de todos los insumos.

In [5]:
solver.Minimize(
    3000*variables[('La Pedrera', 'carne')] + 
    650*variables[('La Pedrera', 'harina')] + 
    1300*variables[('La Pedrera', 'tomate')] +
    1800*variables[('La Pedrera', 'queso')] +
    3200*variables[('El Nogal', 'carne')] + 
    550*variables[('El Nogal', 'harina')] + 
    1200*variables[('El Nogal', 'tomate')] +
    1900*variables[('El Nogal', 'queso')])

status = solver.Solve()

In [6]:
if status == pywraplp.Solver.OPTIMAL:
    print('El problema tiene una solución óptima')
else:
    print('No se encontró una solución óptima')

El problema tiene una solución óptima


In [ ]:
for proveedor in ['La Pedrera', 'El Nogal']:
    for insumo in ['carne', 'harina', 'tomate', 'queso']:
        print(f'{variables[(proveedor, insumo)].name()} = {variables[(proveedor, insumo)].solution_value()}')

- Cantidad de kilos de carne del proveedor La Pedrera = 15.0
- Cantidad de kilos de harina del proveedor La Pedrera = 40.0
- Cantidad de kilos de tomate del proveedor La Pedrera = 0.0
- Cantidad de kilos de queso del proveedor La Pedrera = 25.0
- Cantidad de kilos de carne del proveedor El Nogal = 5.0
- Cantidad de kilos de harina del proveedor El Nogal = 10.0
- Cantidad de kilos de tomate del proveedor El Nogal = 30.0
- Cantidad de kilos de queso del proveedor El Nogal = 5.0

Costo total de los insumos con 80kg

In [8]:
3000*variables[('La Pedrera', 'carne')].solution_value() +650*variables[('La Pedrera', 'harina')].solution_value() +1300*variables[('La Pedrera', 'tomate')].solution_value() +800*variables[('La Pedrera', 'queso')].solution_value() +200*variables[('El Nogal', 'carne')].solution_value() +550*variables[('El Nogal', 'harina')].solution_value() +1200*variables[('El Nogal', 'tomate')].solution_value() +900*variables[('El Nogal', 'queso')].solution_value()

138000.0

Precio sombra que fuerza gastar al menso $60.000 en la distribuidora La Pedrera

In [9]:
acuerdo_pedrera.dual_value()

0.0

# Con camión de 100kg

In [11]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [12]:
variables = {}
for proveedor in ['La Pedrera', 'El Nogal']:
    for insumo in ['carne', 'harina', 'tomate', 'queso']:
        var = solver.NumVar(0, solver.infinity(), f'Cantidad de kilos de {insumo} del proveedor {proveedor}')
        variables[(proveedor, insumo)] = var

In [13]:
for proveedor in ['La Pedrera', 'El Nogal']:
    solver.Add(
        variables[(proveedor, 'carne')] + 
        variables[(proveedor, 'harina')] + 
        variables[(proveedor, 'tomate')] + 
        variables[(proveedor, 'queso')] <= 100, 
        f'Maxima cantidad que puedo comprar al proveedor {proveedor}'
    ) 


solver.Add(variables[('La Pedrera', 'carne')] + variables[('El Nogal', 'carne')] == 20, 'Maxima cantidad de kg de carne total.')
solver.Add(variables[('La Pedrera', 'harina')] + variables[('El Nogal', 'harina')] == 50, 'Maxima cantidad de kg de harina total.')
solver.Add(variables[('La Pedrera', 'tomate')] + variables[('El Nogal', 'tomate')] == 30, 'Maxima cantidad de kg de tomate total.')
solver.Add(variables[('La Pedrera', 'queso')] + variables[('El Nogal', 'queso')] == 30, 'Maxima cantidad de kg de queso total.')


solver.Add(variables[('La Pedrera', 'carne')]  <= 15, 'Maxima cantidad de kg de carne de La Pedrera.')
solver.Add(variables[('El Nogal', 'harina')]  <= 10, 'Maxima cantidad de kg de harina de El Nogal.')

acuerdo_pedrera = solver.Add(
    3000*variables[('La Pedrera', 'carne')] + 
    650*variables[('La Pedrera', 'harina')] + 
    1300*variables[('La Pedrera', 'tomate')] +
    1800*variables[('La Pedrera', 'queso')] >= 60000, 
    'Acuerdo con La Pedrera')

In [14]:
solver.Minimize(
    3000*variables[('La Pedrera', 'carne')] + 
    650*variables[('La Pedrera', 'harina')] + 
    1300*variables[('La Pedrera', 'tomate')] +
    1800*variables[('La Pedrera', 'queso')] +
    3200*variables[('El Nogal', 'carne')] + 
    550*variables[('El Nogal', 'harina')] + 
    1200*variables[('El Nogal', 'tomate')] +
    1900*variables[('El Nogal', 'queso')]
    )

status = solver.Solve()

In [ ]:
for proveedor in ['La Pedrera', 'El Nogal']:
    for insumo in ['carne', 'harina', 'tomate', 'queso']:
        print(f'{variables[(proveedor, insumo)].name()} = {variables[(proveedor, insumo)].solution_value()}')

- Cantidad de kilos de carne del proveedor La Pedrera = 15.0
- Cantidad de kilos de harina del proveedor La Pedrera = 40.0
- Cantidad de kilos de tomate del proveedor La Pedrera = 0.0
- Cantidad de kilos de queso del proveedor La Pedrera = 30.0
- Cantidad de kilos de carne del proveedor El Nogal = 5.0
- Cantidad de kilos de harina del proveedor El Nogal = 10.0
- Cantidad de kilos de tomate del proveedor El Nogal = 30.0
- Cantidad de kilos de queso del proveedor El Nogal = 0.0

Costo total de los insumos con el camión de 100kg.

In [15]:
3000*variables[('La Pedrera', 'carne')].solution_value() +650*variables[('La Pedrera', 'harina')].solution_value() +1300*variables[('La Pedrera', 'tomate')].solution_value() +800*variables[('La Pedrera', 'queso')].solution_value() +200*variables[('El Nogal', 'carne')].solution_value() +550*variables[('El Nogal', 'harina')].solution_value() +1200*variables[('El Nogal', 'tomate')].solution_value() +900*variables[('El Nogal', 'queso')].solution_value()

137500.0

Al aumentar el tamaño del camión, logramos reducir el costo total en $2500. Puesto que el alquiler del camión cuesta $10000, no es recomendable alquilar el camión de mayor tamaño.